Construction d'un schéma exponentiel pour Vlasov-Maxwell hybride linéarisé
===

On regardera ici comment calculer l'exponentielle de matrice nécessaire pour construire un schéma de Lawson.

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from IPython.display import display
sp.init_printing(use_latex='mathjax')

On travaille sur le système :

$$
  \begin{cases}
    \partial_t j_{c,x} = \Omega_{pe}^2E_x - j_{c,y}B_0 \\
    \partial_t j_{c,y} = \Omega_{pe}^2E_y + j_{c,x}B_0 \\
    \partial_t B_x     =  \partial_zE_y \\
    \partial_t B_y     = -\partial_zE_x \\
    \partial_t E_x     = -\partial_zB_y - j_x + \int v_xf\,\mathrm{d}v \\
    \partial_t E_y     =  \partial_zB_x - j_y + \int v_yf\,\mathrm{d}v \\
    \partial_t f       = - v_z\partial_t f + (E_x+v_yB_0-v_zB_y)\partial_{v_x}f + (E_y-v_xB_0+v_zB_x)\partial_{v_y}f + (v_xB_y-v_yB_x)\partial_{v_z}f 
  \end{cases}
$$

Ce qui nous donne, sous forme matricielle (après une transformée de Fourier en $z$ et sans changement dans les notations) :

$$
  \partial_t \begin{pmatrix}
    j_{c,x} \\ j_{c_y} \\ B_x \\ B_y \\ E_x \\ E_y \\ f
  \end{pmatrix}
  =
  \begin{pmatrix}
     0 & -1 & 0  &  0  &  \Omega_{pe}^2 & 0             & 0 \\
     1 &  0 & 0  &  0  &  0             & \Omega_{pe}^2 & 0 \\
     0 &  0 & 0  &  0  &  0             & ik            & 0 \\
     0 &  0 & 0  &  0  & -ik            & 0             & 0 \\
    -1 &  0 & 0  & -ik &  0             & 0             & 0 \\
     0 & -1 & ik &  0  &  0             & 0             & 0 \\
     0 &  0 & 0  &  0  &  0             & 0             & -ikv_z
  \end{pmatrix}
  \begin{pmatrix}
    j_{c,x} \\ j_{c_y} \\ B_x \\ B_y \\ E_x \\ E_y \\ f
  \end{pmatrix}
  +
  \begin{pmatrix}
    0 \\ 0 \\ 0 \\ 0 \\ \int v_xf\,\mathrm{d}v \\ \int v_yf\,\mathrm{d}v \\ (E+v\times(B_0+B))\cdot\partial_vf
  \end{pmatrix}
$$

Sauf qu'il n'est pas possible de calculer l'exponentielle de cette matrice pour tout $\Delta t$ de manière exacte. Il est donc décidé de transférer le transport en $z$ (les "$ik$" dans la matrice) dans le terme non linéaire pour enfin écrire :

$$
  \partial_t U = AU + F(U)
$$

avec $U = (j_{c,x} , j_{c_y} , B_x , B_y , E_x , E_y , f)^\textsf{T}$ et :

$$
  A = \begin{pmatrix}
     0 & -1 & 0  &  0  &  \Omega_{pe}^2 & 0             & 0 \\
     1 &  0 & 0  &  0  &  0             & \Omega_{pe}^2 & 0 \\
     0 &  0 & 0  &  0  &  0             & 0             & 0 \\
     0 &  0 & 0  &  0  &  0             & 0             & 0 \\
    -1 &  0 & 0  &  0  &  0             & 0             & 0 \\
     0 & -1 & 0  &  0  &  0             & 0             & 0 \\
     0 &  0 & 0  &  0  &  0             & 0             & -ikv_z
  \end{pmatrix}
  \quad,\quad
  F(U) = \begin{pmatrix}
    0 \\ 0 \\ ikE_y \\ -ikE_x \\ -ikB_y + \int v_xf\,\mathrm{d}v \\ ikB_x + \int v_yf\,\mathrm{d}v \\ (E+v\times(B_0+B))\cdot\partial_vf
  \end{pmatrix}
$$

In [ ]:
jx,jy = sp.symbols(r"j_{c\,x} j_{c\,y}")
Bx,By = sp.symbols(r"B_x B_y")
Ex,Ey = sp.symbols(r"E_x E_y")
f = sp.symbols(r"\hat{f}")
wpe = sp.symbols(r"\Omega_{pe}",real=True,positive=True)
dt = sp.symbols(r"\Delta\ t",real=True,positive=True)
k = sp.symbols("k")
vx,vy,vz = sp.symbols("v_x v_y v_z",real=True)

U = sp.Matrix([jx,jy,Bx,By,Ex,Ey,f])
U

In [ ]:
A = sp.Matrix([[ 0 , -1 , 0 , 0 , wpe**2 , 0      ,  0         ],
               [ 1 ,  0 , 0 , 0 , 0      , wpe**2 ,  0         ],
               [ 0 ,  0 , 0 , 0 , 0      , 0      ,  0         ],
               [ 0 ,  0 , 0 , 0 , 0      , 0      ,  0         ],
               [-1 ,  0 , 0 , 0 , 0      , 0      ,  0         ],
               [ 0 , -1 , 0 , 0 , 0      , 0      ,  0         ],
               [ 0 ,  0 , 0 , 0 , 0      , 0      , -sp.I*k*vz ]
              ])
A

In [ ]:
eA = sp.exp(A[:-1,:-1]*dt)
eA.subs(wpe,2).evalf().simplify()

Il n'est pas nécessaire de calculer la matrice pour une valeur de $\Omega_{pe}$ quelconque. Cette grandeur est liée à des constantes physiques et on a $\Omega_{pe}=2$.

Pour s'assurer de la non nécessité numérique de calculer pour une valeur quelconque de $\Omega_{pe}$, regardons le premier terme de la matrice.

In [ ]:
eA[0,0]

In [ ]:
Ix = sp.Function(r"\int_\mathbb{R}\ v_x")
Iy = sp.Function(r"\int_\mathbb{R}\ v_y")
df = sp.Function(r"(E+v\times B)\cdot\nabla_v")
def F(U):
  return sp.Matrix([
     0,
     0,
     sp.I*k*U[5],
    -sp.I*k*U[4],
    -sp.I*k*U[3]+Ix(U[6]),
     sp.I*k*U[2]+Iy(U[6]),
     df(U[6])
  ])
F(U)

Pour calculer l'exponentielle de $\Delta t A$ rapidement, je calcule l'exponentielle de la sous matrice (sans la dernière colonne et denière ligne) et je construis ensuite la matrice exponentielle par bloc.

In [ ]:
def expM(A):
  A = A.subs(wpe,2)
  eA = sp.exp(A[:-1,:-1]).evalf().simplify().row_join(sp.zeros(6,1)).col_join(sp.Matrix(1,7,[0]*6+[sp.exp(A[-1,-1])]))
  return eA.simplify()

expM(A*dt)

On écrit maintenant le schéma Lawson induit par la méthode RK(3,3) de Shu-Osher.

In [ ]:
jxn,jyn = sp.symbols(r"j_{c\,x}^n j_{c\,y}^n")
Bxn,Byn = sp.symbols(r"B_x^n B_y^n")
Exn,Eyn = sp.symbols(r"E_x^n E_y^n")
fn = sp.symbols(r"\hat{f}^n")
Un = sp.Matrix([jxn,jyn,Bxn,Byn,Exn,Eyn,fn])

jx1,jy1 = sp.symbols(r"j_{c\,x}^{(1)} j_{c\,y}^{(1)}")
Bx1,By1 = sp.symbols(r"B_x^{(1)} B_y^{(1)}")
Ex1,Ey1 = sp.symbols(r"E_x^{(1)} E_y^{(1)}")
f1 = sp.symbols(r"\hat{f}^{(1)}")
Us1 = sp.Matrix([jx1,jy1,Bx1,By1,Ex1,Ey1,f1])

jx2,jy2 = sp.symbols(r"j_{c\,x}^{(2)} j_{c\,y}^{(2)}")
Bx2,By2 = sp.symbols(r"B_x^{(2)} B_y^{(2)}")
Ex2,Ey2 = sp.symbols(r"E_x^{(2)} E_y^{(2)}")
f2 = sp.symbols(r"\hat{f}^{(2)}")
Us2 = sp.Matrix([jx2,jy2,Bx2,By2,Ex2,Ey2,f2])

U1 = expM(A*dt)*Un + dt*expM(A*dt)*F(Un)
U2 = sp.Rational(3,4)*expM(sp.Rational(1,2)*A*dt)*Un + sp.Rational(1,4)*expM(-sp.Rational(1,2)*dt*A)*Us1 + sp.Rational(1,4)*dt*expM(-sp.Rational(1,2)*dt*A)*F(Us1)
Un1 = sp.Rational(1,3)*expM(A*dt)*Un + sp.Rational(2,3)*expM(sp.Rational(1,2)*dt*A)*Us2 + sp.Rational(2,3)*dt*expM(sp.Rational(1,2)*dt*A)*F(Us2)

display(U1,U2,Un1)

Il faut maintenant remplacer tous les symbols par des expressions qui peuvent être des noms de variable, ainsi que les fonctions. Certaines optimisations, comme le remplacement des divisions par 2 par une multiplication par 0.5, sont également faites.

In [ ]:
def expr_to_code (expr,symbols_replace,function_replace):
  math_to_stl = [(f,"std::"+f) for f in ("sin","cos","exp","sqrt")]

  a = sp.Wild('a')
  def func(expr):
    print(expr is sp.S.Half,expr)
    return sp.Float(0.5)
  
  tmp = expr.subs(symbols_replace)
  for old,new in function_replace:
    tmp = tmp.replace(old,new)
  #tmp = tmp.replace(lambda expr:expr is sp.S.Half,lambda _:sp.Float(0.5)).simplify().evalf()
  tmp = tmp.replace(a/2,0.5*a,exact=True).simplify()
  tmp = tmp.replace(lambda expr:expr is sp.S.One or expr == 1.0,lambda _:sp.S.One).simplify().evalf()
  
  tmp = tmp.replace(lambda expr:expr.is_integer and (expr is not sp.S.One or expr is not sp.S.NegativeOne),lambda expr:sp.symbols(str(expr)+"."))
  stmp = str(tmp)
  #display(gv.Source(sp.dotprint(tmp)))
  for old,new in math_to_stl:
    stmp = stmp.replace(old,new)
  return stmp

In [ ]:
sym_to_code = [(dt,sp.symbols("dt")),(k,sp.symbols("Kz[i]")),
               (Exn,sp.symbols("hEx[i]")),(Eyn,sp.symbols("hEy[i]")),
               (jxn,sp.symbols("hjcx[i]")),(jyn,sp.symbols("hjcy[i]")),
               (Bxn,sp.symbols("hBx[i]")),(Byn,sp.symbols("hBy[i]")),
               (fn,sp.symbols(r"hf[k_x][k_y][k_z][i]")),
               (Ex1,sp.symbols("hEx1[i]")),(Ey1,sp.symbols("hEy1[i]")),
               (jx1,sp.symbols("hjcx1[i]")),(jy1,sp.symbols("hjcy1[i]")),
               (Bx1,sp.symbols("hBx1[i]")),(By1,sp.symbols("hBy1[i]")),
               (f1,sp.symbols(r"hf1[k_x][k_y][k_z][i]")),
               (Ex2,sp.symbols("hEx2[i]")),(Ey2,sp.symbols("hEy2[i]")),
               (jx2,sp.symbols("hjcx2[i]")),(jy2,sp.symbols("hjcy2[i]")),
               (Bx2,sp.symbols("hBx2[i]")),(By2,sp.symbols("hBy2[i]")),
               (f2,sp.symbols(r"hf2[k_x][k_y][k_z][i]")),
               #(sp.sqrt(2),sp.symbols("sqrt2_v"))
              ]
fun_to_code = [(Ix,lambda _:sp.symbols("hjhx[i]")),(Iy,lambda _:sp.symbols("hjhy[i]")),(df,lambda _:sp.symbols("hfvxvyvz[i]"))]

for stage,nextU in zip([U1,U2,Un1],[Us1,Us2,Un]) :
  code_nextU = nextU.subs(sym_to_code)
  for i,l in enumerate(stage):
    print(str(code_nextU[i])+" = "+expr_to_code(l,sym_to_code,fun_to_code)+";")
    print("// ---")
  print("\n/* ------ */\n")

J'ai essayé plusieurs fois de supprimer les `1.0` qui apparaissent dans les formules, mais je n'y suis jamais parvenu.

On teste maintenant du Lawson(RK(4,4))

In [ ]:
jx1,jy1 = sp.symbols(r"j_{c\,x}^{(1)} j_{c\,y}^{(1)}")
Bx1,By1 = sp.symbols(r"B_x^{(1)} B_y^{(1)}")
Ex1,Ey1 = sp.symbols(r"E_x^{(1)} E_y^{(1)}")
f1 = sp.symbols(r"\hat{f}^{(1)}")
Us1 = sp.Matrix([jx1,jy1,Bx1,By1,Ex1,Ey1,f1])

jx2,jy2 = sp.symbols(r"j_{c\,x}^{(2)} j_{c\,y}^{(2)}")
Bx2,By2 = sp.symbols(r"B_x^{(2)} B_y^{(2)}")
Ex2,Ey2 = sp.symbols(r"E_x^{(2)} E_y^{(2)}")
f2 = sp.symbols(r"\hat{f}^{(2)}")
Us2 = sp.Matrix([jx2,jy2,Bx2,By2,Ex2,Ey2,f2])

jx3,jy3 = sp.symbols(r"j_{c\,x}^{(3)} j_{c\,y}^{(3)}")
Bx3,By3 = sp.symbols(r"B_x^{(3)} B_y^{(3)}")
Ex3,Ey3 = sp.symbols(r"E_x^{(3)} E_y^{(3)}")
f3 = sp.symbols(r"\hat{f}^{(3)}")
Us3 = sp.Matrix([jx3,jy3,Bx3,By3,Ex3,Ey3,f3])

U1 = expM(sp.Rational(1,2)*A*dt)*Un + sp.Rational(1,2)*dt*expM(sp.Rational(1,2)*A*dt)*F(Un)
U2 = expM(sp.Rational(1,2)*A*dt)*Un + sp.Rational(1,2)*dt*F(Us1)
U3 = expM(A*dt)*Un + dt*expM(sp.Rational(1,2)*A*dt)*F(Us2)
Un1 = -sp.Rational(1,3)*expM(A*dt)*Un + sp.Rational(1,3)*expM(sp.Rational(1,2)*dt*A)*Us1 + sp.Rational(2,3)*expM(sp.Rational(1,2)*dt*A)*Us2 + sp.Rational(1,3)*Us3 + sp.Rational(1,6)*dt*F(Us3)

display(U1,U2,Us3,Un1)

In [ ]:
sym_to_code = [(dt,sp.symbols("dt")),(k,sp.symbols("Kz[i]")),
               (Exn,sp.symbols("hEx[i]")),(Eyn,sp.symbols("hEy[i]")),
               (jxn,sp.symbols("hjcx[i]")),(jyn,sp.symbols("hjcy[i]")),
               (Bxn,sp.symbols("hBx[i]")),(Byn,sp.symbols("hBy[i]")),
               (fn,sp.symbols(r"hf[k_x][k_y][k_z][i]")),
               (Ex1,sp.symbols("hEx1[i]")),(Ey1,sp.symbols("hEy1[i]")),
               (jx1,sp.symbols("hjcx1[i]")),(jy1,sp.symbols("hjcy1[i]")),
               (Bx1,sp.symbols("hBx1[i]")),(By1,sp.symbols("hBy1[i]")),
               (f1,sp.symbols(r"hf1[k_x][k_y][k_z][i]")),
               (Ex2,sp.symbols("hEx2[i]")),(Ey2,sp.symbols("hEy2[i]")),
               (jx2,sp.symbols("hjcx2[i]")),(jy2,sp.symbols("hjcy2[i]")),
               (Bx2,sp.symbols("hBx2[i]")),(By2,sp.symbols("hBy2[i]")),
               (f2,sp.symbols(r"hf2[k_x][k_y][k_z][i]")),
               (Ex3,sp.symbols("hEx3[i]")),(Ey3,sp.symbols("hEy3[i]")),
               (jx3,sp.symbols("hjcx3[i]")),(jy3,sp.symbols("hjcy3[i]")),
               (Bx3,sp.symbols("hBx3[i]")),(By3,sp.symbols("hBy3[i]")),
               (f3,sp.symbols(r"hf3[k_x][k_y][k_z][i]")),
               #(sp.sqrt(2),sp.symbols("sqrt2_v"))
              ]
fun_to_code = [(Ix,lambda _:sp.symbols("hjhx[i]")),(Iy,lambda _:sp.symbols("hjhy[i]")),(df,lambda _:sp.symbols("hfvxvyvz[i]"))]

for stage,nextU in zip([U1,U2,U3,Un1],[Us1,Us2,Us3,Un]) :
  code_nextU = nextU.subs(sym_to_code)
  for i,l in enumerate(stage):
    print(str(code_nextU[i])+" = "+expr_to_code(l,sym_to_code,fun_to_code)+";")
    print("// ---")
  print("\n/* ------ */\n")

Dans le code généré il est encore nécessaire d'écrire les boucles qui vont autour, de la manière suivante :

1. Calcul des variables $\hat{j}_{c,x}^{(1)}$, $\hat{j}_{c,y}^{(1)}$, $\hat{E}_x^{(1)}$, $\hat{E}_y^{(1)}$, $\hat{B}_x^{(1)}$ et $\hat{B}_y^{(1)}$ :
    + calcul des intégrales $\int v_x \hat{f}\,\mathrm{d}v$ et $\int v_y \hat{f}\,\mathrm{d}v$ :
        - $\left(\hat{j}_{h,x}\right)_i \gets \sum_{k_x,k_y,k_z} v_{k_x}\,\hat{f}_{i,k_x,k_y,k_z}\,\Delta v$
        - $\left(\hat{j}_{h,y}\right)_i \gets \sum_{k_x,k_y,k_z} v_{k_y}\,\hat{f}_{i,k_x,k_y,k_z}\,\Delta v$
    + pour tout $x_i$ (code donné par `sympy`) :
        - $\hat{j}_{c,x}^{(1)}\gets \cdots$
        - $\hat{j}_{c,y}^{(1)}\gets \cdots$
        - $\hat{E}_{c,x}^{(1)}\gets \cdots$
        - $\hat{E}_{c,y}^{(1)}\gets \cdots$
        - $\hat{B}_{c,x}^{(1)}\gets \cdots$
        - $\hat{B}_{c,y}^{(1)}\gets \cdots$
2. Calcul de la variable $\hat{f}^{(1)}$ :
    + transformée inverse de $\hat{f}^{n}$ :
        - $\left(f\right)_{i,k_x,k_z,k_z} \gets \text{iFFT}(\hat{f}^n_{\cdot,k_x,k_z,k_z})_i$ (boucle en $v_x$, $v_y$ et $v_z$)
    + calcul de l'approximation de $(E_x+v_yB_0 + v_zB_y)\partial_{v_x}f + (E_y-v_xB_0+v_zB_x)\partial_{v_y}f +(v_xB_y - v_yB_x)\partial_{v_z}f$ :
        - pour tout $v_{k_x}$ :
            - pour tout $v_{k_y}$ :
                - pour tout $v_{k_y}$ :
                    - pour tout $v_{k_z}$ :
                        - pour tout $k_i$ :
                            - $\texttt{velocity_vx} \gets E_{x,i}+v_{k_y}B_0 + v_{k_z}B_{y,i}$
                            - $\texttt{velocity_vy} \gets E_{y,i}-v_{k_x}B_0 + v_{k_z}B_{x,i}$
                            - $\texttt{velocity_vz} \gets v_{k_x}B_{y,i} - v_{k_y}B_{x,i}$
                            - $\partial_vf_{i,k_x,k_y,k_z}\gets \text{WENO}(\texttt{velocity_vx},f_{i,k_x-3:k_x+3,k_y,k_z}) + \text{WENO}(\texttt{velocity_vy},f_{i,k_x,k_y-3:k_y+3,k_z}) + \text{WENO}(\texttt{velocity_vz},f_{i,k_x,k_y,k_z-3:k_z+3})$
    + incrémentation de $\hat{f}^{(1)}$ :
        - pour tout $v_{k_x}$ :
            - pour tout $v_{k_y}$ :
                - pour tout $v_{k_y}$ :
                    - pour tout $v_{k_z}$ :
                        - $\left(\widehat{\partial_vf}\right)_i\gets\text{FFT}(\partial_vf_{\cdot,k_x,k_y,k_z})_i$
                        - pour tout $k_i$ :
                            - $\hat{f}^{(1)}_{i,k_x,k_y,k_z} \gets \hat{f}^n_{i,k_x,k_y,k_z} + \Delta t\widehat{\partial_vf}_i$

où $\text{WENO}$ est une fonction qui calcule une approximation de $a\partial_xu(x_i)$ en prenant en argument $a$ et le stencil $u_{i-3:i+3}$. Cette fonction WENO diffère de ce que j'avais déjà pu implémenter avant, car il s'agissait alors d'une fonction renvoyant le flux numérique $\hat{f}^\pm_{i+\frac{1}{2}}$. C'est-à-dire $\text{WENO}(a,u_{i-3},u_{i-2},u_{i-1},u_{i},u_{i+1},u_{i+2},u_{i+3})$ est une fonction renvoyant l'approximation : $$a^+\frac{u^+_{i+\frac{1}{2}}-u^+_{i-\frac{1}{2}}}{\Delta x} + a^-\frac{u^-_{i+\frac{1}{2}}-u^-_{i-\frac{1}{2}}}{\Delta x}$$ donc les flux numériques sont calculés 2 fois, ce qui engendre des pertes en temps de calcul, mais un gain important en utilisation de la mémoire (puisqu'il n'est pas nécessaire de sauvergarder tous les flux numériques avant calcul des différences).

Il existe un module `codegen` dans `sympy` qui ressemble à ce que je fais, mais celui-ci génère des fonctions, suppose que le symbole est directement convertible en nom de variable, etc. et fait toujours apparaître des `1.0` un peu partout dans ses expressions. Par exemple pour chaque "ligne" d'une méthode d'Euler explicite.

In [ ]:
from sympy.utilities.codegen import codegen

for l in expM(A*dt)*Un + dt*expM(A*dt)*F(U) :
  print(codegen(("osef",l),"C")[0][1])
  print("---")

Il ne serait pas inintéressant de s'intéresser un peu plus au module, mais je ne sais pas comment définir proprement un tableau (car il faut remplacer tous les symbols par des tableaux) avec `sympy`... donc à voir...

Ce module présente l'avantage d'utiliser les variables du module `math.h` de `C` (par exemple `M_SQRT2`). Ces variables ont leur équivalent `C++` uniquement à partir de `C++20` dans le module `cmath`, avant il faut utiliser ceux de `C`.

> **Nota Bene :** j'ai remarqué récemment qu'il était possible d'indiquer que tous mes tableaux sont des objets indicés (`sp.IndexedBase`) et cela permet, lors de la génération de code, de dérouler les boucles. Mais vu la lourdeur d'utilisation pour moi, par rapport à ce que j'ai pu coder... je pense que je ne vais pas explorer plus cette piste tout de suite.

### une énième tentative de mettre le plus de choses dans la partie linéaire

In [ ]:
B = sp.Matrix([[ 0 , -1 , 0      ,  0      ,  wpe**2 , 0      ,  0         ],
               [ 1 ,  0 , 0      ,  0      ,  0      , wpe**2 ,  0         ],
               [ 0 ,  0 , 0      ,  0      ,  0      , sp.I*k ,  0         ],
               [ 0 ,  0 , 0      ,  0      , -sp.I*k , 0      ,  0         ],
               [-1 ,  0 , 0      , -sp.I*k ,  0      , 0      ,  0         ],
               [ 0 , -1 , sp.I*k ,  0      ,  0      , 0      ,  0         ],
               [ 0 ,  0 , 0      ,  0      ,  0      , 0      , -sp.I*k*vz ]
              ])
try:
  sp.exp(B.subs(wpe,2)*dt)
except(NotImplementedError):
  print("sympy gère pas cette matrice")

In [ ]:
for e in (B[:-1,:-1].subs(wpe,2)*dt).eigenvals():
  display(e)
  print("---")

Les valeurs propres de la sous-matrice (sans le transport en $z$ de $f$) sont relativement compliquées, il est donc très compliqué d'obtenir l'exponentielle de la matrice. `sympy` semble malgré tout calculer quelque chose, donc il est peut-être envisageable d'effectuer le calcul sur le serveur de calcul... à voir...

Une méthode itérative est aussi envisageable pour avoir une méthode approchée, mais je ne sais pas si cela est vraiment possible avec du calcul formel (nécessaire pour le faire pour tout mode $k$ et pour tout $\Delta t$).

Les tentatives de calcul direct pour tout mode $k$ et tout $\Delta t>0$ sur le serveur (donc sans contrainte de temps de calcul) ont échoué, même en fixant le mode $k$ (donc en voulant faire une trentaine de calcul pour chaque mode de Fourier dans la discrétisation en $z$).